<a href="https://colab.research.google.com/github/longaresf/Machine_learning_simulacro_examen_1/blob/main/Francisco_Longares_Clase_4_30_06_25_simulacro_examen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Simulacro

Entrena un modelo que aprenda a predecir la columna income en este dataset.

In [1]:
import pandas as pd
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
df_csv = pd.read_csv(url, header=None, na_values=" ?", skipinitialspace=True)
column_names = [
    "age",
    "workclass",
    "fnlwgt",
    "education",
    "education_num",
    "marital_status",
    "occupation",
    "relationship",
    "race",
    "sex",
    "capital_gain",
    "capital_loss",
    "hours_per_week",
    "native_country",
    "income"
]
df_csv.columns = column_names

Diccionario de columnas:

| Columna          | Tipo                        | Descripción                                                                      |
| ---------------- | --------------------------- | -------------------------------------------------------------------------------- |
| `age`            | Numérica                    | Edad del individuo (años)                                                        |
| `workclass`      | Categórica nominal          | Tipo de empleador (por ejemplo, `Private`, `Self-emp`, `Government`, etc.)       |
| `fnlwgt`         | Numérica                    | Peso de la muestra (valor demográfico asignado por el censo)                     |
| `education`      | Categórica ordinal          | Nivel de educación (por ejemplo, `Bachelors`, `HS-grad`, `Masters`, etc.)        |
| `education_num`  | Numérica                    | Nivel de educación codificado como número (más alto = más educación)             |
| `marital_status` | Categórica nominal          | Estado civil (por ejemplo, `Married`, `Divorced`, `Never-married`, etc.)         |
| `occupation`     | Categórica nominal          | Ocupación (por ejemplo, `Tech-support`, `Craft-repair`, `Sales`, etc.)           |
| `relationship`   | Categórica nominal          | Relación familiar dentro del hogar (por ejemplo, `Wife`, `Husband`, `Own-child`) |
| `race`           | Categórica nominal          | Raza del individuo (por ejemplo, `White`, `Black`, `Asian-Pac-Islander`, etc.)   |
| `sex`            | Categórica nominal          | Sexo del individuo (`Male`, `Female`)                                            |
| `capital_gain`   | Numérica                    | Ganancia de capital (dinero ganado por inversiones, etc.)                        |
| `capital_loss`   | Numérica                    | Pérdida de capital (dinero perdido por inversiones, etc.)                        |
| `hours_per_week` | Numérica                    | Horas trabajadas por semana                                                      |
| `native_country` | Categórica nominal          | País de origen del individuo                                                     |
| `income`         | Categórica binaria (target) | Clase objetivo: `<=50K` o `>50K`, según ingreso anual                            |


In [2]:
df_csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education_num   32561 non-null  int64 
 5   marital_status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital_gain    32561 non-null  int64 
 11  capital_loss    32561 non-null  int64 
 12  hours_per_week  32561 non-null  int64 
 13  native_country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [39]:
# Librerias a importar
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Parte 1: Análisis Exploratorio de Datos (EDA)
## 1. Carga y Exploración de Datos:
Descripción de las variables y su distribución.

In [40]:
# Exploración de datos
print(df_csv.head())
print(df_csv.describe())

   age         workclass  fnlwgt  education  education_num  \
0   39         State-gov   77516  Bachelors             13   
1   50  Self-emp-not-inc   83311  Bachelors             13   
2   38           Private  215646    HS-grad              9   
3   53           Private  234721       11th              7   
4   28           Private  338409  Bachelors             13   

       marital_status         occupation   relationship   race     sex  \
0       Never-married       Adm-clerical  Not-in-family  White    Male   
1  Married-civ-spouse    Exec-managerial        Husband  White    Male   
2            Divorced  Handlers-cleaners  Not-in-family  White    Male   
3  Married-civ-spouse  Handlers-cleaners        Husband  Black    Male   
4  Married-civ-spouse     Prof-specialty           Wife  Black  Female   

   capital_gain  capital_loss  hours_per_week native_country income  
0          2174             0              40  United-States  <=50K  
1             0             0             

Detección y eliminación de valores duplicados

In [41]:
# Hacemos copia del DataFrame
df=df_csv.copy()
# Buscar datos duplicados
datos_duplicados = df.duplicated()
if datos_duplicados.any():
    print("Cantidad de datos duplicados: ", datos_duplicados.sum())
else:
    print("No se encontró datos duplicados")

Cantidad de datos duplicados:  24


In [42]:
# Validar el shape.
print(df.shape)

(32561, 15)


In [43]:
# Eliminar datos duplicados.
df.drop_duplicates(inplace=True)
# Validar el shape.
print(df.shape)

(32537, 15)


Verificamos tipo de dato de columnas.<br>
Consistencia en valores categóricos

In [44]:
# Verificamos valores únicos en las columas
for col in df.columns:
    print(f"\nDatos en la columa: '{col}':")
    print(df[col].unique())


Datos en la columa: 'age':
[39 50 38 53 28 37 49 52 31 42 30 23 32 40 34 25 43 54 35 59 56 19 20 45
 22 48 21 24 57 44 41 29 18 47 46 36 79 27 67 33 76 17 55 61 70 64 71 68
 66 51 58 26 60 90 75 65 77 62 63 80 72 74 69 73 81 78 88 82 83 84 85 86
 87]

Datos en la columa: 'workclass':
['State-gov' 'Self-emp-not-inc' 'Private' 'Federal-gov' 'Local-gov' '?'
 'Self-emp-inc' 'Without-pay' 'Never-worked']

Datos en la columa: 'fnlwgt':
[ 77516  83311 215646 ...  34066  84661 257302]

Datos en la columa: 'education':
['Bachelors' 'HS-grad' '11th' 'Masters' '9th' 'Some-college' 'Assoc-acdm'
 'Assoc-voc' '7th-8th' 'Doctorate' 'Prof-school' '5th-6th' '10th'
 '1st-4th' 'Preschool' '12th']

Datos en la columa: 'education_num':
[13  9  7 14  5 10 12 11  4 16 15  3  6  2  1  8]

Datos en la columa: 'marital_status':
['Never-married' 'Married-civ-spouse' 'Divorced' 'Married-spouse-absent'
 'Separated' 'Married-AF-spouse' 'Widowed']

Datos en la columa: 'occupation':
['Adm-clerical' 'Exec-managerial'

In [46]:
# Verificamos los datos en las columas
for col in df.columns:
    print(f"\nRevisamos Datos en la columa: '{col}':")
    print(df[col].value_counts())


Revisamos Datos en la columa: 'age':
age
36    898
31    888
34    886
23    876
35    875
     ... 
83      6
88      3
85      3
86      1
87      1
Name: count, Length: 73, dtype: int64

Revisamos Datos en la columa: 'workclass':
workclass
Private             22673
Self-emp-not-inc     2540
Local-gov            2093
?                    1836
State-gov            1298
Self-emp-inc         1116
Federal-gov           960
Without-pay            14
Never-worked            7
Name: count, dtype: int64

Revisamos Datos en la columa: 'fnlwgt':
fnlwgt
123011    13
203488    13
164190    13
126675    12
148995    12
          ..
148751     1
212302     1
169885     1
112780     1
327112     1
Name: count, Length: 21648, dtype: int64

Revisamos Datos en la columa: 'education':
education
HS-grad         10494
Some-college     7282
Bachelors        5353
Masters          1722
Assoc-voc        1382
11th             1175
Assoc-acdm       1067
10th              933
7th-8th           645
Prof-school 

Detección de datos anómalos

In [47]:
# Detectar valores extremos o fuera de rango segun interpretacion de las columnas
df.describe()

,age,fnlwgt,education_num,capital_gain,capital_loss,hours_per_week
count,32537.000000,3.253700e+04,32537.000000,32537.000000,32537.000000,32537.000000
mean,38.585549,1.897808e+05,10.081815,1078.443741,87.368227,40.440329
std,13.637984,1.055565e+05,2.571633,7387.957424,403.101833,12.346889
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.369930e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [48]:
# Usamos .loc para seleccionar las filas de columnas con valores '?'
# y asignarles el valor NaN.

columnas = ['native_country','occupation','workclass']

for col in columnas:
    df.loc[df[col] == '?', col] = np.nan
    # Verificamos si se eliminó el valor de '?' en la columna.
    print(f"\nCantidad de '?' después de la corrección en la columa: '{col}':")
    print(f"{df[df[col] == '?'].shape[0]}")
    # Verificamos la cantidad de datos NaN en las columnas.
    print(f"Cantidad de datos 'NaN' en columna: '{col}' \n{df[df[col].isna()].shape[0]}")


Cantidad de '?' después de la corrección en la columa: 'native_country':
0
Cantidad de datos 'NaN' en columna: 'native_country' 
582

Cantidad de '?' después de la corrección en la columa: 'occupation':
0
Cantidad de datos 'NaN' en columna: 'occupation' 
1843

Cantidad de '?' después de la corrección en la columa: 'workclass':
0
Cantidad de datos 'NaN' en columna: 'workclass' 
1836


Uso de ColumnTransformer y Pipeline

In [49]:
# Cargar el conjunto de datos
X = df.drop('income', axis=1)
y = df['income']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Definir las columnas numéricas y categóricas
numeric_features = ['age', 'fnlwgt', 'education_num', 'capital_gain', 'capital_loss', 'hours_per_week']
categorical_features = ['workclass', 'education', 'marital_status', 'occupation', 'relationship', 'race', 'sex', 'native_country']

# Crear transformadores para cada subconjunto de columnas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# Crear un ColumnTransformer
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
])

# Crear un Pipeline completo
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

# Ajustar el Pipeline a los datos de entrenamiento
pipeline.fit(X_train, y_train)

# Hacer predicciones utilizando el Pipeline
y_pred = pipeline.predict(X_test)

# Calcular la precisión del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Precisión del modelo: {accuracy:.2f}")

Precisión del modelo: 0.86
